Dataloader for Cifar dataset


In [31]:
import torch
import torchvision
import torchvision.transforms as transforms

class CifarLoader(object):
    """docstring for CifarLoader"""
    def __init__(self):
        super(CifarLoader, self).__init__()
        transform = transforms.Compose(
		    [
		     # TODO: Add data augmentations here
             transforms.RandomHorizontalFlip(),
		     transforms.ToTensor(),
             transforms.Normalize((0.5,),(0.5,))
		     ])
        transform_test = transforms.Compose([
		    transforms.ToTensor(),
            transforms.Normalize((0.5,),(0.5,))
            #transforms.Normalize((0.5),(0.2))
		])
  
        train_set = torchvision.datasets.MNIST('./', transform=transform, download=True)
        print(len(train_set))
        self.trainloader = torch.utils.data.DataLoader(train_set,
                                              batch_size=64,
                                              shuffle=True)
        test_set = torchvision.datasets.MNIST('./', transform=transform_test, download=True)
        self.testloader = torch.utils.data.DataLoader(test_set,
                                              batch_size=64,
                                              shuffle=False)    

Network architecture class.
net = Net() will initialize a network. The architecture is defined in the forward() function.
Calling the network to perform one step of prediction is simply net(input), this will give input to the network and call the forward() function.

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


Training and testing function.
You need to filling in the missing steps in this function

In [33]:
def train(net, dataloader, optimizer, criterion, epoch):

    running_loss = 0.0
    total_loss = 0.0

    for i, data in enumerate(dataloader.trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + compute loss + backward + update weights
        # 1. forward pass of network
        output = net(inputs)
        # 2. Compute loss use criterion function
        loss = criterion(output, labels)
        # 3. compute gradient with respect to loss use backpropagation
        loss.backward()
        # 4. update network weights use optimizer
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()
        if (i + 1) % 2000 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    print('Final Summary:   loss: %.3f' %
          (total_loss / i))


def test(net, dataloader, tag=''):
    correct = 0
    total = 0
    if tag == 'Train':
        dataTestLoader = dataloader.trainloader
    else:
        dataTestLoader = dataloader.testloader
    with torch.no_grad():
        for data in dataTestLoader:
            images, labels = data
            # 1. forward pass of network
            output = net(images)
            # 2. Prediction is the maximum label in the output
            _,prediction = torch.max(output.data,1)
            # 3. Compare with the groundtruth label and count the number of 
            #    correct predictions
            total += labels.size(0)
            correct += (prediction == labels).sum().item()


    print('%s Accuracy of the network: %d %%' % (tag,
        100 * correct / total))


Training and testing network

In [35]:

import torch.optim as optim

epochs = 5  # 
net = Net()
print(net)

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.05)
criterion = nn.NLLLoss()

# Create dataloader
cifarLoader = CifarLoader()

#train(net, cifarLoader, optimizer, criterion, 1)
for epoch in range(epochs):  # loop over the dataset multiple times
    train(net, cifarLoader, optimizer, criterion, epoch)
    if epoch % 1 == 0: # Comment out this part if you want a faster training
        test(net, cifarLoader, 'Train')
        test(net, cifarLoader, 'Test')

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
60000
Final Summary:   loss: 0.832
Train Accuracy of the network: 87 %
Test Accuracy of the network: 87 %
Final Summary:   loss: 0.192
Train Accuracy of the network: 95 %
Test Accuracy of the network: 95 %
Final Summary:   loss: 0.123
Train Accuracy of the network: 96 %
Test Accuracy of the network: 96 %
Final Summary:   loss: 0.096
Train Accuracy of the network: 97 %
Test Accuracy of the network: 97 %
Final Summary:   loss: 0.078
Train Accuracy of the network: 97 %
Test Accuracy of the network: 97 %
